In [71]:
import pandas as pd
import os
import re
from collections import deque
from openpyxl.styles import PatternFill

# Colors for highlighting
GREEN_FILL = PatternFill(start_color="77DD76", end_color="77DD76", fill_type="solid")
RED_FILL = PatternFill(start_color="FFB6B3", end_color="FFB6B3", fill_type="solid")
RED2_FILL = PatternFill(start_color="FF6962", end_color="FF6962", fill_type="solid")

def process_transaction(row, balances):
    global all_transactions
    # Convert amount to integer (cents)
    amount = int(row['Kwota'])
    multiple_lines = False

    # Find exchange rate in the title using regex
    match = re.search(r'eFX kurs: ([34]\.\d{4})', row['Tytuł'])
    rate = float(match.group(1)) if match else None

    if amount > 0:  # Inflow
        if rate is None:
            print(f"Warning {row['Data księgowania']}: No exchange rate found for transaction: {row['Tytuł']}")
            row['kurs'] = '????'
            balances.append((amount, '????'))
        else:
            balances.append((amount, rate))
            row['kurs'] = rate
        all_transactions.append(row)
        print("przychodzaca kwota: ", amount, row['kurs'], "stan: ", balances)

    else:  # Outflow
        remaining = -amount
        original_row = row.copy()
        original_row['kurs'] = None
        split_rows = []

        while remaining > 0 and balances:
            inflow_amount, rate = balances.popleft()

            if rate != '????':
                if remaining <= inflow_amount and multiple_lines == False:
                    # Fully processed within one rate
                    print(f"wychodzaca kwota cała: {remaining}, {rate}, stan: {balances}")
                    original_row['kurs'] = rate
                    if remaining < inflow_amount:
                        balances.appendleft((inflow_amount - remaining, rate))
                    remaining = 0
                elif inflow_amount >= remaining:
                    # last inflow
                    balances.appendleft((inflow_amount - remaining, rate))
                    print(f"wychodząca kwota ostatnia, {remaining}, {rate}, stan: {balances}")
                    split_rows.append(pd.Series({'Kwota': -remaining, 'Waluta': row['Waluta'], 'kurs': rate}))
                    remaining = 0
                    multiple_lines = True
                else:
                    # partially processed inflow
                    split_row = pd.Series({'Kwota': -inflow_amount, 'Waluta': row['Waluta'], 'kurs': rate})
                    split_rows.append(split_row)
                    remaining -= inflow_amount
                    print(f"wychodząca kwota częściowa, {inflow_amount}, {rate}, stan: {balances}")
                    multiple_lines = True

            else: # rate == '????'
                if remaining <= inflow_amount:
                    print(f"Outflow '????' fully processed: {remaining}, stan: {balances}")
                    original_row['kurs'] = '????'
                    if inflow_amount > remaining:
                        balances.appendleft((inflow_amount - remaining, rate))
                    remaining = 0
                else:
                    # Partially processed for unknown rate
                    split_row = pd.Series({'Kwota': -inflow_amount, 'Waluta': row['Waluta'], 'kurs': '????'})
                    split_rows.append(split_row)
                    remaining -= inflow_amount
                    print(f"wychodząca kwota częściowa, {inflow_amount}, {rate}, stan: {balances}, zostało {remaining}")

        if remaining > 0:
            print(f"Outflow not fully processed: {remaining}, stan: {balances}")
            split_row = pd.Series({'Kwota': -remaining, 'Waluta': row['Waluta'], 'kurs': '????'})
            split_rows.append(split_row)

        if split_rows:
            all_transactions.append(original_row)
            for row in split_rows:
                all_transactions.append(row)
        else:
            all_transactions.append(original_row)

def print_summary(balances):
    print("\nUSD Balance:")
    print(f"Remaining inflows: {[(amount, rate) for amount, rate in balances]}")

# Load Excel file
def load_transactions(file_name):
    df = pd.read_excel(file_name)
    df['kurs'] = None
    df['Kwota'] = df['Kwota']*100
    return df

def save_transactions(file_name, transactions):
    from openpyxl import Workbook
    from openpyxl.utils.dataframe import dataframe_to_rows

    # Create a new workbook
    wb = Workbook()
    ws = wb.active

    # Write data to the worksheet
    for r_idx, row in enumerate(dataframe_to_rows(transactions, index=False, header=True), start=1):
        for c_idx, value in enumerate(row, start=1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)

            # Highlight rows based on amount
            if r_idx > 1:  # Skip header
                if transactions.iloc[r_idx - 2]['Kwota'] > 0:
                    cell.fill = GREEN_FILL
                elif transactions.iloc[r_idx - 2]['Kwota'] < 0:
                    if pd.isna(transactions.iloc[r_idx - 2]['Typ operacji']):
                        cell.fill = RED_FILL
                    else:
                        cell.fill = RED2_FILL

    wb.save(file_name)


### USD

In [ ]:

balances = deque()
total = 0
all_transactions = []
def main():
    input_file = os.path.join('waluty', 'usd.xlsx')
    transactions = load_transactions(input_file)
    transactions.sort_values(by=['Data księgowania', 'Kwota'], ascending=[True, False], inplace=True)
    transactions['Data księgowania'] = pd.to_datetime(transactions['Data księgowania'], format='%Y-%m-%d')
    transactions = transactions[transactions['Data księgowania'] <= '2024-10-31']

    for _, row in transactions.iterrows():
        if row['Waluta'] == 'USD':
            process_transaction(row, balances)

    # Save transactions to the another file
    save_transactions(input_file.replace('.xlsx', '_output.xlsx'), pd.DataFrame(all_transactions))

    # Print summary
    print_summary(balances)

if __name__ == "__main__":
    main()

### EUR

In [ ]:
balances = deque()
total = 0
all_transactions = []

def main():
    
    input_file = os.path.join('waluty', 'euro.xlsx')
    transactions = load_transactions(input_file)
    transactions.sort_values(by=['Data księgowania', 'Kwota'], ascending=[True, False], inplace=True)
    transactions['Data księgowania'] = pd.to_datetime(transactions['Data księgowania'], format='%Y-%m-%d')
    transactions = transactions[transactions['Data księgowania'] <= '2024-10-31']
    print(transactions)

    for _, row in transactions.iterrows():
        if row['Waluta'] == 'EUR':
            process_transaction(row, balances)

    # Save transactions to the another file
    save_transactions(input_file.replace('.xlsx', '_output.xlsx'), pd.DataFrame(all_transactions))

    # Print summary
    print_summary(balances)

if __name__ == "__main__":
    main()